## 任务6：多路召回+重排序case2

## BM25 (gensim)
#### 教学

In [47]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel,OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np

In [48]:
corpus =[
    "今天 很开心",
    "明天 也 很开心",
    "Python is programming language"
]
corpus

['今天 很开心', '明天 也 很开心', 'Python is programming language']

In [49]:
corpus_split = [doc.lower().split() for doc in corpus]
corpus_split

[['今天', '很开心'],
 ['明天', '也', '很开心'],
 ['python', 'is', 'programming', 'language']]

In [50]:
dictionary=Dictionary(corpus_split)  ## 这个是将里面的每个value进行了去重，生成词典
for key,value in dictionary.items():
    print(key,value)

0 今天
1 很开心
2 也
3 明天
4 is
5 language
6 programming
7 python


In [51]:
bm25_model = OkapiBM25Model(dictionary=dictionary) # 相当于给模型喂一本词典

In [52]:


# dictionary.doc2bow：是一个方法，用来将文档表示为词袋的形式（Bag of Words），即将文档中的单词转换成整数索引，并计算它们的词频
# map(dictionary.doc2bow,corpus_split)：对corpus_split中的每个文档应用转换成词袋的表示形式
# list是将这些词袋表示的文档重新组合成一个列表
# bm25_model[...] 这部分代码使用了 Python 中的列表推导式，对 list(map(dictionary.doc2bow, corpus_split)) 中的每个文档应用了 bm25_model，即将文档转换为 Okapi BM25 模型的表示形式。
# bm25_model[list(map(dictionary.doc2bow, corpus_split))] 这行代码实际上是将整个语料库（由 corpus_split 表示）转换为 Okapi BM25 模型的表示形式，以便后续用于相似性计算和搜索

bm25_corpus = bm25_model[list(map(dictionary.doc2bow,corpus_split))] # 先转换成词袋，再转换成模型的表示形式
# for i in bm25_corpus:
#     print(i)

In [53]:
# bm25_corpus 是已经通过 Okapi BM25 模型处理过的语料库，其中每个文档都被表示为一个 Okapi BM25 模型表示形式。
# SparseMatrixSimilarity 是 Gensim 库中的类，用于构建一个稀疏矩阵相似性索引器。
# num_docs=len(corpus_split) 指定了语料库中文档的数量，这里使用了 len(corpus_split) 来获取文档数量。
# num_terms=len(dictionary) 指定了语料库中的唯一词汇数量，这里使用了 len(dictionary) 来获取词汇数量。
# normalize_queries=False 表示不对查询进行规范化，即不对查询向量进行归一化处理。
# normalize_documents=False 表示不对文档进行规范化，即不对文档向量进行归一化处理。
# 因此，整个行代码的含义是在基于 Okapi BM25 模型处理过的语料库上构建一个稀疏矩阵相似性索引器 bm25_index，以便后续用于执行相似性搜索操作。这个索引器可以用于计算查询与文档之间的相似性得分，从而找到与查询最匹配的文档。
# 生成了相似性索引器
bm25_index = SparseMatrixSimilarity(bm25_corpus,num_docs=len(corpus_split),num_terms=len(dictionary),normalize_queries=False,normalize_documents=False)


In [54]:
query = ["learn","今天"]
tfidf_model=TfidfModel(dictionary=dictionary,smartirs='bnn') # 使用之前的 字典，并bnn指定了对查询进行二进制加权
tfidf_query = tfidf_model[dictionary.doc2bow(query)] #将查询词转换成词袋的表示，并进行TF-IDF加权 
smiilarities = bm25_index[tfidf_query]  # 用相似性索引器 计算查询文本tfidf_query与语料库中的每个文档的相似性得分
print(smiilarities) # 打印出与每个文档的相似性得分
best_document=corpus_split[np.argmax(smiilarities)] # 找出相似性得分最大的，并且输出索引，最后查询出词
print(best_document)

[0.60097134 0.         0.        ]
['今天', '很开心']


## BM25 (gensim)
#### 真实数据

In [55]:
# 1. 导入模块
from datasets import load_dataset
import jieba
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score

In [56]:
# 2. 导入数据
data = load_dataset('C-MTEB/DuRetrieval')
qrels = load_dataset('C-MTEB/DuRetrieval-qrels')

In [57]:
# 3. 展示数据
data # 数据集中的2000条问题和100001条知识库

DatasetDict({
    corpus: Dataset({
        features: ['id', 'text'],
        num_rows: 100001
    })
    queries: Dataset({
        features: ['id', 'text'],
        num_rows: 2000
    })
})

In [58]:
qrels # 问题对应答案的索引

DatasetDict({
    dev: Dataset({
        features: ['qid', 'pid', 'score'],
        num_rows: 9839
    })
})

In [35]:
corpus_show = pd.DataFrame(data["corpus"]) # 答案text 和id
corpus_show[:5] 

,id,text
0,2c4fe63d3378ac39907b6b2648eb40c5,一年国家法定节假日为11天。根据公布的国家法定节假日调整方案，调整的主要内容包括：元旦放假1...
1,5bc347ff17d488f1704e2893c9e8ecfa,"【导读】根据现行我国《全国年节及纪念日放假办法》规定,全体公民放假的法定节假日总共有11天,..."
2,1132b79d0bc37fc0d6f975904a8c0343,答：国家规定的2016年法定节假日累计有11天。2016年法定假日一览表元旦：规定在1月1日...
3,6e67389d07da8ce02ed97167d23baf9d,现在法定假日是元旦1天，春节3天，清明节1天，五一劳动节1天，端午节1天，国庆节3天，中秋节...
4,ac02549bbcd642d8d43321b94cd9b62b,国家法定有薪假11天。根据国务院《全国年节及纪念日放假办法》规定，国家法定全体公民有薪假11...


In [36]:
queries_show = pd.DataFrame(data["queries"]) # 问题text 和id
queries_show[:5] 

,id,text
0,edb58f525bd14724d6f490722fa8a657,国家法定节假日共多少天
1,a451acd1e9836b04b16664e9f0c290e5,如何查看好友申请
2,48a8338aefaff17573048a088a08de70,哪个网站有湖南卫视直播
3,7871706c5cb1a1d6912ff8222434ccbd,功和功率的区别
4,2b22b972c4e30776b9160fc83ee05367,徐州旅行社哪家好


In [37]:
# 4. 知识库处理
corpus = [jieba.lcut(x['text']) for x in data["corpus"]] # 生成答案，知识库

In [38]:
# 5. 知识库相似度模型处理
dictionary=Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus=bm25_model[list(map(dictionary.doc2bow,corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus,num_docs=len(corpus),num_terms=len(dictionary),normalize_documents=False,normalize_queries=False)


In [39]:
# 6. 问题模型
tfidf_model=TfidfModel(dictionary=dictionary,smartirs='afc')

In [40]:
# 7. 对问题求相似度
%%time
query="请注意识别，谨防上当受骗！" # 对当前问题查询出相似度最高的知识库中的句子
query = jieba.lcut(query)
tfidf_query=tfidf_model[dictionary.doc2bow(query)]
similarities = bm25_index[tfidf_query]
best_document = corpus[np.argmax(similarities)]
" ".join(best_document)

CPU times: total: 15.6 ms
Wall time: 15.6 ms


'该 经验 图片 、 文字 中 可能 存在 外站 链接 或 电话号码 等 , 请 注意 识别 , 谨防 上当受骗 ! 百度 经验 : jingyan . baidu . com 在 之前 win7 的 设置 中 大部分 操作 都 是 在 控制面板 中 完成 的 , 可是 在 新 的 win10 系统 中 却 发现 控制面板 很难 找 了 , 那么 在 哪里 能 找到 呢 ? 让 我 来 告诉 你 吧 ~ 百度 经验 : jingyan . baidu . com 百度 经验 : jingyan . baidu . com1   打开 开始菜单 , 点击 所有 应用   步骤 阅读   2   找到 windows 系统 , 点击 打开 下级菜单   步骤 阅读   3   在 当中 找到 控制面板 点击 打开 即可   步骤 阅读   END 百度 经验 : jingyan . baidu . com'

In [41]:
# 1. 将问题与答案映射id处理成字典
qrels_dict={x['qid']:x["pid"] for x in qrels["dev"]} # 问题与答案的id，index
pid_array = np.array(x["id"] for x in data["corpus"]) # 答案的id

In [42]:
# 2. 直接导入rank_bm25模型
from rank_bm25 import BM25Okapi
tokenized_corpus = [jieba.lcut(x["text"]) for x in data["corpus"]]
bm25 = BM25Okapi(tokenized_corpus)

tfidf_model=TfidfModel(dictionary=dictionary,smartirs='afc')

In [59]:
# 3. 准确度计算
topn = 30
query_ndcg_score = []
for query_data in tqdm(data['queries']):
    query = jieba.lcut(query_data['text']) # 问题
    
    query_qid = query_data['id'] # 问题id
    query_pids = qrels_dict[query_qid] # 对应的答案id

    # 计算相似度矩阵的两种方式
    similarities = np.array(bm25.get_scores(query))
    # similarities=bm25_index[tfidf_query]

    top_results = similarities.argsort()[::-1][:topn]
    top_results = [data['corpus'][int(x)]['id'] for x in top_results]
    true_relevance  = [[x in query_pids for x in top_results]]
    scores = [list(similarities[similarities.argsort()[::-1][:topn]])]

    query_ndcg_score.append(ndcg_score(true_relevance, scores))

100%|██████████| 2000/2000 [03:58<00:00,  8.39it/s]


In [60]:
np.mean(query_ndcg_score), np.std(query_ndcg_score)

(0.3449331376092457, 0.3423558128037693)